In [1]:
#Active Library dependencies
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyphysio as ph
from pdb import set_trace
import random
#Always plot inline if possible
%matplotlib inline

Using dask. Scheduler: threads
Please cite:
Bizzego et al. (2019) 'pyphysio: A physiological signal processing library for data science approaches in physiology', SoftwareX


In [7]:
#Preprocessing filters that will be applied to raw data samples
def exp_moving_average(signal, w):
    """Expoential moving average filter from pandas"""
    return pd.Series(signal.ewm(span=w, adjust=True).mean(), signal.index)

def filt_EDA(df_data):
    """Apply filter to EDA signal using processing steps from Pyphysio.
    
    Parameters
    ----------
    df_data : pandas.DataFrame
        DataFrame with EDA time and signal data
    
    Returns
    -------
    pandas.DataFrame 
        Updated DataFrame with Tonic and Phasic signals
    
    """
    
    # Set up Pyphysio EvenlySignal object
    eda_data = ph.EvenlySignal(values = df_data['EDA'].to_numpy(), sampling_freq = 4, signal_type='EDA')
    
    # Apply IIR filter
    eda_data = ph.IIRFilter(fp=0.8, fs=1.1, ftype='ellip')(eda_data)
    driver = ph.DriverEstim()(eda_data)
    
    # Estimate Tonic and Phasic signals
    phasic, tonic, _ = ph.PhasicEstim(delta=0.02)(driver)
    
    # Adjust signal length to match length of original EDA signal
    if len(phasic) != len(eda_data.get_values()):
        phasic = np.append(phasic.get_values(), phasic[-1])
        tonic = np.append(tonic.get_values(), tonic[-1])    
    
    # Append signal data to DataFrame
    df_data.loc[:,'Tonic'] = tonic
    df_data.loc[:,'Phasic'] = phasic
    return df_data  

def filt_TEMP(df_data):
    """Apply filter to TEMP signal using predetermined values"""
    df_data['TEMP_Filtered'] = exp_moving_average(df_data['TEMP'],60)
    return df_data

def filter_signals(df_data):
    """Apply filters/processing to respective signals.
    
    This function is intended to be used in conjunction with the
    pandas.DataFrame.apply() method, which passes a column of a DataFrame at
    a time.
    
    Parameters
    ----------
    df_data : pandas.Series
        A Series of DataFrames which contain all the signals for a single
        session.
    
    Returns
    -------
    pandas.Series
        Updated Series of DataFrames containing filtered/processed signal
        data
        
    Notes
    -----
    No filtering/processing for ACC, HR, and IBI signals is implemented,
    but is commented out for implementation in the future.
    
    """
    
    #df_data.loc['BVP'].loc[:,'BVP'] = filt_BVP(df_data.loc['BVP'])
    df_data = filt_EDA(df_data)
    df_data = filt_TEMP(df_data)
    
    return df_data

In [15]:
#RMS calculation Helper Function
def rms(data):
    return np.sqrt(np.mean(data ** 2))

#Every other feature is easy to calculate using Python built-ins
def feature_extract(df_data):
    result = {}
    result['Time'] = df_data.loc[:,'timestamp'].min()
    #for featbase in ['HR','EDA','TEMP','Tonic','Phasic','TEMP_Filtered']:
    for featbase in ['HR','EDA','TEMP']:
        #set_trace()
        result[featbase + '_Mean'] = df_data.loc[:,featbase].mean()
        result[featbase + '_Minimum'] = df_data.loc[:,featbase].min()
        result[featbase + '_Maximum'] = df_data.loc[:,featbase].max()
        result[featbase + '_Stdev'] = df_data.loc[:,featbase].std()
        result[featbase + '_RMS'] = rms(df_data.loc[:,featbase])
        result[featbase + '_MAD'] = df_data.loc[:,featbase].mad()
        result[featbase + '_MAV'] = df_data.loc[:,featbase].abs().max()
        result[featbase + '_Median'] = df_data.loc[:,featbase].median()
        result[featbase + '_P25'] = df_data.loc[:,featbase].quantile(0.25)
        result[featbase + '_P75'] = df_data.loc[:,featbase].quantile(0.75)
        result['pid'] = behavior_df['PID']
    return pd.Series(result, dtype='object')


In [3]:
os.chdir('/Users/sandoval/Library/CloudStorage/Box-Box/R15 Sensor Preprocessing and Analysis/10 Minute Windows/')

In [4]:
#Load baseline and behavior data for ALL participants into a dataframe
#participants = ['PR021','PR022','PR023','PR032','PR037','PR038','PR039','PR042','PR044','PR059','PR073','PR074','PR078']
participants = ['PR021','PR022','PR023','PR032','PR037','PR038','PR039','PR044','PR059','PR073','PR074','PR078']

behavior_df = pd.DataFrame()
baseline_df = pd.DataFrame()

for curpar in participants:
    #First load the behavior windows
    behfile = curpar + '/'+ curpar + ' Behavior 40 min Windows MERGED.csv'
    cur_behdf = pd.read_csv(behfile,parse_dates=['timestamp'], infer_datetime_format=True)
    cur_behdf['PID'] = curpar 
    if behavior_df.empty:
        behavior_df = cur_behdf
    else:
        behavior_df = pd.concat([behavior_df, cur_behdf])
    #Now load the baseline windows
    basefile = curpar + '/'+ curpar + ' Baseline MERGED.csv'
    cur_basedf = pd.read_csv(basefile,parse_dates=['timestamp'], infer_datetime_format=True)
    cur_basedf['PID'] = curpar 
    if baseline_df.empty:
        baseline_df = cur_basedf
    else:
        baseline_df = pd.concat([baseline_df, cur_basedf])
    

In [5]:
behavior_df

,timer,timestamp,HR,X,Y,Z,TEMP,BVP,EDA,event,code,PID
0,0.25,2020-11-21 16:33:12.250,78.95,14.0,-11.0,61.0,31.81,6.18,0.012806,1,1,PR021
1,0.50,2020-11-21 16:33:12.500,78.95,5.0,-11.0,62.0,31.81,-3.52,0.012806,1,1,PR021
2,0.75,2020-11-21 16:33:12.750,78.95,18.0,-12.0,60.0,31.81,25.98,0.012806,1,1,PR021
3,1.00,2020-11-21 16:33:13.000,78.95,15.0,-11.0,61.0,31.81,63.54,0.011526,1,1,PR021
4,1.25,2020-11-21 16:33:13.250,78.97,16.0,-11.0,60.0,31.81,-227.75,0.012806,1,1,PR021
...,...,...,...,...,...,...,...,...,...,...,...,...
38383,2397.00,2021-09-25 04:58:40.000,124.68,-79.0,-3.0,16.0,33.91,-1.88,0.127111,4,1,PR078
38384,2397.25,2021-09-25 04:58:40.250,124.73,-38.0,-30.0,-18.0,33.91,60.53,0.119424,4,1,PR078
38385,2397.50,2021-09-25 04:58:40.500,124.73,-16.0,18.0,-70.0,33.91,-179.37,0.101486,4,1,PR078
38386,2397.75,2021-09-25 04:58:40.750,124.73,-31.0,-19.0,-57.0,33.91,213.15,0.080986,4,1,PR078


In [28]:
physio_base_pr032 = pd.read_csv("/Users/sandoval/Library/CloudStorage/Box-Box/R15 Sensor Preprocessing and Analysis/10 Minute Windows/PR003/PR003 Behavior 40 min Windows MERGED.csv",parse_dates=['timestamp'], infer_datetime_format=True, index_col=[0])

In [31]:
physio = physio_base_pr032.sort_values(['timestamp'], ignore_index=True)
physio = physio.groupby(['event']).apply(feature_extract)
physio['pid'] = 'PR032'
print(physio)

                         Time     HR_Mean  HR_Minimum  HR_Maximum   HR_Stdev  \
event                                                                          
1     2020-11-09 12:57:59.250   98.004984        1.00      181.62  27.693846   
2     2020-11-09 22:11:24.750   73.645720       47.43      104.12  14.760919   
3     2020-11-10 12:05:05.500   95.286813       63.77      121.08  13.087836   
4     2020-11-11 11:41:00.250   69.088693        1.00      105.18  10.719415   
5     2020-11-12 00:46:25.000   57.551184       45.90       73.55   6.835620   
6     2020-11-12 12:58:40.250   85.285723        1.00      115.73  11.814063   
7     2020-11-13 00:20:30.750   75.874470       53.28      115.38  13.188508   
8     2020-11-13 12:32:37.500   95.171283       53.22      145.77  19.943067   
9     2020-11-15 00:38:05.000   68.106902       50.85      118.77  15.402228   
10    2020-11-16 12:13:43.750  103.358516       75.58      122.78  14.016802   
11    2020-11-17 12:02:37.500   93.10442

In [32]:
for col in physio.columns:
    print(col)

Time
HR_Mean
HR_Minimum
HR_Maximum
HR_Stdev
HR_RMS
HR_MAD
HR_MAV
HR_Median
HR_P25
HR_P75
pid
EDA_Mean
EDA_Minimum
EDA_Maximum
EDA_Stdev
EDA_RMS
EDA_MAD
EDA_MAV
EDA_Median
EDA_P25
EDA_P75
TEMP_Mean
TEMP_Minimum
TEMP_Maximum
TEMP_Stdev
TEMP_RMS
TEMP_MAD
TEMP_MAV
TEMP_Median
TEMP_P25
TEMP_P75


In [33]:
physio['pid']

event
1     PR032
2     PR032
3     PR032
4     PR032
5     PR032
6     PR032
7     PR032
8     PR032
9     PR032
10    PR032
11    PR032
12    PR032
13    PR032
14    PR032
15    PR032
16    PR032
Name: pid, dtype: object